<a href="https://colab.research.google.com/github/silverstar0727/study-/blob/master/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 신경망(Neural Network)
torch.nn 패키지를 이용하여 신경망을 생성하며, 여기서 앞서 배운 autograd를 사용한다. nn.Module은 layer와 output을 반환하는 forward메서드를 포함한다.

mnist에서 숫자 이미지를 분류하는 신경망의 예제를 통해 공부해보자.

우선 신경망의 일반적인 학습과정은 다음과 같다
* 학습 가능한 매개변수를 갖는 신경망을 정의한다.
* 데이터셋 입력을 반복한다.
* 입력을 신경망에서 전파(process)한다.
* 손실(loss)에 대해 계산한다
* gradient를 신경망의 배개변수들에 역전파한다
* 신경망의 가중치를 갱신한다(weight = prev_weight - learning rate * gradient)

In [ ]:
# 신경망 정의하기
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): # nn.Module을 상속받음
  def __init__(self):
    super(Net, self).__init__()
    # 1개의 input image channel, 6개의 output channels, 3by3 square convolution
    # kernel
    self.conv1 = nn.Conv2d(1, 6, 3)
    self.conv2 = nn.Conv2d(6, 16, 3)
    # an affine operation: y = Wx + b
    self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6by6 from image dimension
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    # Max pooling over a (2, 2) window
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    # window가 정사각형이라면 하나의 숫자로 특정할 수 있다
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:] # batch 차원을 제외한 모든 차원
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
# 학습 가능한 매개변수는 net.parameters()에 의해 반환
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [ ]:
# 32by32 데이터 셋을 입력
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0680,  0.1066, -0.1343, -0.0233,  0.0344,  0.1344, -0.0366,  0.0966,
          0.0643,  0.0446]], grad_fn=<AddmmBackward>)


In [ ]:
# 모든 매개변수의 변화도 버퍼를 0으로 하고 무작위 값으로 역전파 진행
net.zero_grad()
out.backward(torch.randn(1, 10))


## Loss Function
간단한 손실 함수로 MSE(mean-squared error)를 nn.MSEloss가 지원

In [ ]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9374, grad_fn=<MseLossBackward>)


## 연산 순서

      input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

In [ ]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## 역전파((back propagation)
loss.backward()를 통해 역전파를 진행하는데, 기존의 변화도를 없애지 않으면 변화가 누적되기 때문에 없애야 한다.

In [ ]:
net.zero_grad() # 모든 파라미터에 대한 gradient buffer를 0으로 조정

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0185,  0.0143,  0.0046, -0.0002, -0.0012,  0.0109])


## 가중치 갱신
간단한 모델의 SGD(Stochastic Gradient Descent)를 활용


In [ ]:
# 직접 구현
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

In [ ]:
# torch.optim패키지를 이용
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)

# training loop
optimizer.zero_grad() # zero the gradient buffer(수동으로 해야됨)
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # does the update